# 🍏 Observability & Tracing Demo with `azure-ai-projects` and `azure-ai-inference` 🍎

> 📚 **For developers and learners**: Refer to the official Azure AI Foundry observability documentation: [https://learn.microsoft.com/en-us/azure/ai-foundry/concepts/observability](https://learn.microsoft.com/en-us/azure/ai-foundry/concepts/observability)

Welcome to this **Health & Fitness**-themed notebook, where we'll explore how to set up **observability** and **tracing** for:

1. **Basic LLM calls** using an `AIProjectClient`.
2. **Multi-step** interactions using an **Agent** (such as a Health Resource Agent).
3. **Tracing** your local usage in **console** (stdout) or via an **OTLP endpoint** (like **Prompty** or **Aspire**).
4. Sending those **traces** to **Azure Monitor** (Application Insights) so you can view them in **Azure AI Foundry**.

> **Disclaimer**: This is a fun demonstration of AI and observability! Any references to workouts, diets, or health routines in the code or prompts are purely for **educational** purposes. Always consult a professional for health advice.

## Contents
1. **Initialization**: Setting up environment, creating clients.
2. **Basic LLM Call**: Quick demonstration of retrieving model completions.
3. **Connections**: Listing project connections.
4. **Observability & Tracing**
   - **Azure Monitor** tracing: hooking up to Application Insights
   - **Verifying** your traces in Azure AI Foundry
5. **Agent-based Example**:
   - Creating a simple "Health Resource Agent" referencing sample docs.
   - Multi-turn conversation with tracing.
   - Cleanup.

<img src="./seq-diagrams/1-observability.png" width="75%"/>

## 🔐 Authentication Setup

Before running the next cell, make sure you're authenticated with Azure CLI. Run this command in your terminal:

```bash
az login --use-device-code
```

This will provide you with a device code and URL to authenticate in your browser, which is useful for:
- Remote development environments
- Systems without a default browser
- Corporate environments with strict security policies

After successful authentication, you can proceed with the notebook cells below.

## 1. Initialization & Setup
**Prerequisites**:
- A `.env` file containing `AI_FOUNDRY_PROJECT_ENDPOINT` (and optionally `MODEL_DEPLOYMENT_NAME`).
- Roles/permissions in Azure AI Foundry that let you do inference & agent creation.
- A local environment with `azure-ai-projects`, `azure-ai-inference`, `opentelemetry` packages installed.

**What we do**:
- Load environment variables.
- Initialize `AIProjectClient`.
- Check that we can talk to a model (like `gpt-4o`).

In [ ]:
import os
import sys
import time
from pathlib import Path
from dotenv import load_dotenv
from azure.identity import InteractiveBrowserCredential
from azure.ai.projects import AIProjectClient
from azure.ai.inference.models import UserMessage, CompletionsFinishReason

# Load environment variables
notebook_path = Path().absolute()
env_path = notebook_path.parent.parent / '.env'  # Adjust path as needed
load_dotenv(env_path)

project_endpoint = os.environ.get("AI_FOUNDRY_PROJECT_ENDPOINT")
tenant_id = os.environ.get("TENANT_ID")
if not project_endpoint:
    raise ValueError("🚨 AI_FOUNDRY_PROJECT_ENDPOINT not set in .env.")

print(f"🔑 Using Tenant ID: {tenant_id}")

# Initialize AIProjectClient with simplified browser-based authentication
try:
    print("🌐 Using browser-based authentication to bypass Azure CLI cache issues...")
    
    # Use only InteractiveBrowserCredential with the specific tenant
    credential = InteractiveBrowserCredential(tenant_id=tenant_id)
    
    # Create the project client using endpoint
    project_client = AIProjectClient(
        endpoint=project_endpoint,
        credential=credential
    )
    print("✅ Successfully created AIProjectClient!")
except Exception as e:
    print(f"❌ Error creating AIProjectClient: {e}")
    print("💡 Please complete the browser authentication prompt that should appear")

## 2. Basic LLM Call
We'll do a **quick** chat completion request to confirm everything is working. We'll ask a simple question: "How many feet are in a mile?"

In [ ]:
from azure.ai.inference.models import UserMessage

model_deployment_name = os.getenv("MODEL_DEPLOYMENT_NAME")

try:
    # Use the correct Azure AI Projects SDK pattern
    print("🔄 Getting OpenAI client from Azure AI Project...")
    print(f"🤖 Using model: {model_deployment_name}")
    
    # Get OpenAI client using the correct method
    openai_client = project_client.get_openai_client(api_version="2024-10-21")
    
    # Create chat completion using OpenAI client pattern
    response = openai_client.chat.completions.create(
        model=model_deployment_name,
        messages=[
            {"role": "system", "content": "You are a helpful health assistant"},
            {"role": "user", "content": "How to be healthy in one sentence?"}
        ]
    )
    
    print("✅ Successfully created chat completion!")
    print(f"🤖 Assistant: {response.choices[0].message.content}")
    
except Exception as e:
    print(f"❌ An error occurred: {str(e)}")
    print("💡 Troubleshooting tips:")
    print("  - Ensure your Azure AI Project has OpenAI connections configured")
    print("  - Verify your MODEL_DEPLOYMENT_NAME is correctly deployed")
    print("  - Check that you have proper permissions to access the model")
    print("  - Make sure you're using the latest azure-ai-projects SDK version")

## 3. Observability & Tracing

In [ ]:
# # Install packages exactly as specified in Microsoft documentation if not installed using requirements.txt
# !pip install azure-ai-projects azure-monitor-opentelemetry opentelemetry-instrumentation-openai-v2

## 3.1 Set Environment Variables for Content Capture

According to Microsoft documentation, we need to set the environment variable `AZURE_TRACING_GEN_AI_CONTENT_RECORDING_ENABLED` to capture message content (prompts and responses). This should be set **before** any instrumentation.

**Important**: This may contain personal data, so use with caution in production environments.

In [ ]:
import os

# Set environment variable to enable content recording BEFORE instrumentation
# This follows the official Microsoft documentation pattern
print("🔧 Setting up environment variables for tracing...")
os.environ["AZURE_TRACING_GEN_AI_CONTENT_RECORDING_ENABLED"] = "true"
print("✅ Content recording enabled for traces (prompts and responses will be captured)")
print("⚠️  Note: This may contain personal data - use with caution in production")

## 3.2 Configure Azure Monitor Tracing

Following the official Microsoft documentation, we will:
1. Get the Application Insights connection string from the project
2. Configure Azure Monitor using `configure_azure_monitor()`
3. Instrument the OpenAI SDK using `OpenAIInstrumentor()`

This setup ensures all traces are sent to Azure AI Foundry's Tracing tab.

In [ ]:
from azure.monitor.opentelemetry import configure_azure_monitor
from opentelemetry.instrumentation.openai_v2 import OpenAIInstrumentor

print("🔧 Setting up Azure Monitor tracing for AI Foundry...")

try:
    # Step 1: Get the Application Insights connection string from the project
    connection_string = project_client.telemetry.get_application_insights_connection_string()
    
    if connection_string:
        print("✅ Retrieved Application Insights connection string")
        
        # Step 2: Configure Azure Monitor with the connection string (Microsoft official pattern)
        configure_azure_monitor(connection_string=connection_string)
        print("✅ Azure Monitor configured successfully")
        
        # Step 3: Instrument the OpenAI SDK (Microsoft official pattern)
        OpenAIInstrumentor().instrument()
        print("✅ OpenAI SDK instrumented for tracing")
        
        print("\n🎯 Tracing is now active! All API calls will be sent to Azure AI Foundry.")
        print("📊 View traces at: https://ai.azure.com -> Your Project -> Tracing")
        
    else:
        print("❌ No Application Insights connection string found")
        print("💡 Please ensure your AI Foundry project has Application Insights connected")
        print("   Go to: Azure AI Foundry Portal -> Your Project -> Tracing -> Enable tracing")
        
except Exception as e:
    print(f"❌ Failed to configure Azure Monitor: {e}")
    print("💡 Troubleshooting:")
    print("   1. Ensure Application Insights is connected to your AI Foundry project")
    print("   2. Check you have proper permissions (Contributor role)")
    print("   3. Verify the project endpoint is correct in your .env file")

## 3.3 Test Basic Tracing with OpenAI Call

Now let's make a simple OpenAI call to verify that tracing is working. This trace should appear in the Azure AI Foundry Tracing tab within a few minutes.

In [ ]:
# Make a simple OpenAI call to test tracing
print("🧪 Testing tracing with a simple OpenAI call...")

try:
    client = project_client.get_openai_client(api_version="2024-10-21")
    response = client.chat.completions.create(
        model=os.environ.get("MODEL_DEPLOYMENT_NAME", "gpt-4o"),
        messages=[
            {"role": "user", "content": "Write a short poem on open telemetry."}
        ]
    )
    
    print("✅ OpenAI call completed successfully!")
    print(f"\n🤖 Response:\n{response.choices[0].message.content}\n")
    print("🔍 This interaction should now be visible in Azure AI Foundry Tracing tab")
    print("⏱️  Note: Traces may take 2-5 minutes to appear in the portal")
    
except Exception as e:
    print(f"❌ Error during test call: {e}")

## 3.4 Optional: Console Tracing for Local Debugging

If you want to see traces in your local console output (useful for debugging), you can set up a console exporter. This is in addition to Azure Monitor tracing.

In [ ]:
# Optional: Set up console tracing for local debugging
from opentelemetry import trace
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import SimpleSpanProcessor, ConsoleSpanExporter

print("🔧 Setting up console tracing for local debugging...")

try:
    # Get the current tracer provider (already configured by Azure Monitor)
    tracer_provider = trace.get_tracer_provider()
    
    # Add console exporter to see traces in stdout
    console_exporter = ConsoleSpanExporter()
    console_processor = SimpleSpanProcessor(console_exporter)
    
    # Add the processor to the existing tracer provider
    if hasattr(tracer_provider, 'add_span_processor'):
        tracer_provider.add_span_processor(console_processor)
        print("✅ Console tracing enabled - traces will also print to console")
    else:
        print("⚠️  Console tracing not available with current tracer provider")
        
except Exception as e:
    print(f"⚠️  Could not set up console tracing: {e}")
    print("💡 This is optional - Azure Monitor tracing should still work")

# Test with console tracing
print("\n🧪 Testing with console tracing enabled...")

try:
    client = project_client.get_openai_client(api_version="2024-10-21")
    response = client.chat.completions.create(
        model=os.environ.get("MODEL_DEPLOYMENT_NAME", "gpt-4o"),
        messages=[
            {"role": "user", "content": "What's a simple 5-minute warmup routine?"}
        ]
    )
    
    print(f"\n🤖 Response: {response.choices[0].message.content}")
    print("\n🔍 Check console output above for detailed trace spans")
    
except Exception as e:
    print(f"❌ Error: {e}")

# 5. Agent Tracing with Azure AI Agents SDK

Following the official Microsoft documentation for tracing agents, we'll now demonstrate:
1. **Setting up agent instrumentation** using `AIAgentsInstrumentor`
2. Creating a **Health Resource Agent** with file search capabilities
3. **Tracing agent operations** with proper span creation
4. Viewing traces in the **Azure AI Foundry portal**

> The agent tracing approach follows the official pattern from: https://learn.microsoft.com/en-us/azure/ai-foundry/how-to/develop/trace-agents-sdk

## 5.1 Enable Agent Instrumentation

Following Microsoft's official documentation, we need to instrument the Azure AI Agents SDK to enable tracing for agent operations.

In [ ]:
# Enable agent instrumentation following Microsoft's official pattern
from azure.ai.agents.telemetry import AIAgentsInstrumentor
from opentelemetry import trace

print("🔧 Setting up agent instrumentation...")

# Instrument the Azure AI Agents SDK (Microsoft official pattern)
AIAgentsInstrumentor().instrument()
print("✅ Azure AI Agents SDK instrumented for tracing")

# Get tracer for creating custom spans
tracer = trace.get_tracer(__name__)
print("✅ Tracer ready for creating custom spans")

print("\n🎯 Agent operations will now be traced and sent to Azure AI Foundry")

## 5.2 Create Sample Files & Vector Store

We'll create dummy `.md` files about recipes/guidelines, then push them into a **vector store** so our agent can do semantic search.

In [ ]:
def create_sample_files():
    """Create some local .md files with sample text."""
    recipes_md = (
        "# Healthy Recipes Database\n\n"
        "## Gluten-Free Recipes\n"
        "1. Quinoa Bowl\n"
        "   - Ingredients: quinoa, vegetables, olive oil\n"
        "   - Instructions: Cook quinoa, add vegetables\n\n"
        "2. Rice Pasta\n"
        "   - Ingredients: rice pasta, mixed vegetables\n"
        "   - Instructions: Boil pasta, sauté vegetables\n\n"
        "## Diabetic-Friendly Recipes\n"
        "1. Low-Carb Stir Fry\n"
        "   - Ingredients: chicken, vegetables, tamari sauce\n"
        "   - Instructions: Cook chicken, add vegetables\n\n"
        "## Heart-Healthy Recipes\n"
        "1. Baked Salmon\n"
        "   - Ingredients: salmon, lemon, herbs\n"
        "   - Instructions: Season salmon, bake\n\n"
        "2. Mediterranean Bowl\n"
        "   - Ingredients: chickpeas, vegetables, tahini\n"
        "   - Instructions: Combine ingredients\n"
    )

    guidelines_md = (
        "# Dietary Guidelines\n\n"
        "## General Guidelines\n"
        "- Eat a variety of foods\n"
        "- Control portion sizes\n"
        "- Stay hydrated\n\n"
        "## Special Diets\n"
        "1. Gluten-Free Diet\n"
        "   - Avoid wheat, barley, rye\n"
        "   - Focus on naturally gluten-free foods\n\n"
        "2. Diabetic Diet\n"
        "   - Monitor carbohydrate intake\n"
        "   - Choose low glycemic foods\n\n"
        "3. Heart-Healthy Diet\n"
        "   - Limit saturated fats\n"
        "   - Choose lean proteins\n"
    )

    with open("recipes.md", "w", encoding="utf-8") as f:
        f.write(recipes_md)
    with open("guidelines.md", "w", encoding="utf-8") as f:
        f.write(guidelines_md)

    print("📄 Created sample resource files: recipes.md, guidelines.md")
    return ["recipes.md", "guidelines.md"]

def create_vector_store(files, store_name="my_health_resources"):
    try:
        uploaded_ids = []
        for fp in files:
            upl = project_client.agents.files.upload_and_poll(
                file_path=fp,
                purpose="assistants"
            )
            uploaded_ids.append(upl.id)
            print(f"✅ Uploaded: {fp} -> File ID: {upl.id}")

        # Create vector store from these file IDs
        vs = project_client.agents.vector_stores.create_and_poll(
            file_ids=uploaded_ids,
            name=store_name
        )
        print(f"🎉 Created vector store '{store_name}', ID: {vs.id}")
        return vs, uploaded_ids
    except Exception as e:
        print(f"❌ Error creating vector store: {e}")
        return None, []

# Create files and vector store
sample_files = create_sample_files()
vector_store, file_ids = None, []

if sample_files:
    vector_store, file_ids = create_vector_store(sample_files, store_name="health_resources_example")

## 5.3 Create Health Resource Agent with Tracing

We'll create an agent with file search capabilities. Following Microsoft's official pattern, we wrap the agent creation in a traced span.

In [ ]:
def create_health_agent(vs_id):
    """Create an agent with file search capabilities, wrapped in a traced span."""
    try:
        # Wrap agent creation in a traced span (Microsoft official pattern)
        with tracer.start_as_current_span("create_agent") as span:
            span.set_attribute("agent.model", os.environ.get("MODEL_DEPLOYMENT_NAME", "gpt-4o"))
            span.set_attribute("agent.name", "health-search-agent")
            span.set_attribute("vector_store.id", vs_id)
            
            agent = project_client.agents.create_agent(
                model=os.environ.get("MODEL_DEPLOYMENT_NAME", "gpt-4o"),
                name="health-search-agent",
                instructions="""
                    You are a health resource advisor with access to dietary and recipe files.
                    You:
                    1. Always present disclaimers (you're not a medical professional)
                    2. Provide references to files when possible
                    3. Focus on general nutrition or recipe tips.
                    4. Encourage professional consultation for more detailed advice.
                """,
                tools=[{"type": "file_search"}]
            )
            
            span.set_attribute("agent.id", agent.id)
            print(f"🎉 Created agent '{agent.name}' with ID: {agent.id}")
            print("📋 Vector store will be attached at message level")
            
            return agent
            
    except Exception as e:
        print(f"❌ Error creating health agent: {e}")
        return None

# Create agent with tracing
health_agent = None
if vector_store:
    health_agent = create_health_agent(vector_store.id)

## 5.4 Run Agent with Tracing

Following Microsoft's official pattern from the documentation, we wrap the entire agent execution in a traced span. This ensures all agent operations are captured and sent to Azure AI Foundry.

In [ ]:
# Run agent with proper tracing (Microsoft official pattern)
if health_agent and file_ids:
    print("🚀 Running agent with tracing enabled...\n")
    
    # Wrap the entire agent session in a traced span (Microsoft official pattern)
    with tracer.start_as_current_span("example-tracing") as span:
        span.set_attribute("agent.id", health_agent.id)
        span.set_attribute("agent.name", health_agent.name)
        
        # Create thread
        thread = project_client.agents.threads.create()
        span.set_attribute("thread.id", thread.id)
        print(f"📝 Created thread: {thread.id}\n")
        
        # Ask a question - we'll attach files via tool_resources instead
        user_question = "Could you suggest a gluten-free lunch recipe?"
        print(f"❓ User: {user_question}\n")
        
        # Update thread with tool resources (vector store)
        project_client.agents.threads.update(
            thread_id=thread.id,
            tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}}
        )
        
        # Create message without attachments - file search will use thread's tool_resources
        message = project_client.agents.messages.create(
            thread_id=thread.id,
            role="user",
            content=user_question
        )
        
        # Run the agent (this is automatically traced by AIAgentsInstrumentor)
        run = project_client.agents.runs.create_and_process(
            thread_id=thread.id,
            agent_id=health_agent.id
        )
        
        span.set_attribute("run.id", run.id)
        span.set_attribute("run.status", run.status)
        print(f"✅ Run completed with status: {run.status}\n")
        
        # Get the agent's response
        messages = project_client.agents.messages.list(thread_id=thread.id)
        message_list = list(messages)
        
        for msg in reversed(message_list):
            if msg.role == "assistant" and msg.content:
                last_content = msg.content[-1]
                if hasattr(last_content, "text"):
                    print(f"🤖 Assistant:\n{last_content.text.value}\n")
                    break
    
    print("✅ Agent execution completed!")
    print("\n🎯 Check Azure AI Foundry Tracing tab for:")
    print("   - Operation name: 'example-tracing'")
    print("   - Agent operations and tool calls")
    print("   - Message exchanges and file search results")
    print("\n⏱️  Traces should appear within 2-5 minutes")
else:
    print("⚠️  Skipping agent execution - agent or files not available")

## 5.5 View Traces in Azure AI Foundry

After running the agent, you can view the traces in the Azure AI Foundry portal:

1. Navigate to https://ai.azure.com
2. Open your project
3. Click on "Tracing" in the left sidebar
4. Look for traces with operation name: **"example-tracing"**

**What you'll see in the traces:**
- The top-level span "example-tracing" containing all operations
- Agent creation span with model and configuration details
- Thread creation and message operations
- Agent run execution with tool calls (file search)
- LLM calls made by the agent
- Input/output data for each operation (if content recording is enabled)

**Troubleshooting:**
- Traces may take 2-5 minutes to appear
- Ensure Application Insights is connected to your project
- Check that `AZURE_TRACING_GEN_AI_CONTENT_RECORDING_ENABLED=true` is set
- Verify the connection string was retrieved successfully earlier

# 6. Cleanup
If desired, we can remove the vector store, files, and agent to keep things tidy. (In a real solution, you might keep them around.)

In [ ]:
# Cleanup resources to keep things tidy
def cleanup_resources():
    """Clean up agent, vector store, and files created during this demo."""
    try:
        print("🧹 Starting cleanup...")
        
        # Delete vector store
        if 'vector_store' in globals() and vector_store:
            project_client.agents.vector_stores.delete(vector_store.id)
            print("✅ Deleted vector store")

        # Delete uploaded files
        if 'file_ids' in globals() and file_ids:
            for fid in file_ids:
                project_client.agents.files.delete(fid)
            print(f"✅ Deleted {len(file_ids)} uploaded files")

        # Delete agent
        if 'health_agent' in globals() and health_agent:
            project_client.agents.delete_agent(health_agent.id)
            print("✅ Deleted agent")

        # Delete local sample files
        if 'sample_files' in globals() and sample_files:
            for sf in sample_files:
                if os.path.exists(sf):
                    os.remove(sf)
            print(f"✅ Deleted {len(sample_files)} local sample files")
            
        print("\n🎉 Cleanup completed successfully!")
        
    except Exception as e:
        print(f"❌ Error during cleanup: {e}")
        print("💡 Some resources may need to be cleaned up manually in Azure AI Foundry portal")

# Uncomment the line below to run cleanup
# cleanup_resources()

# 🎉 Summary

Congratulations! You've successfully set up **observability and tracing** for Azure AI applications!

## What You Learned

1. **Basic Tracing Setup**: 
   - Set environment variables for content capture
   - Configure Azure Monitor with Application Insights connection string
   - Instrument OpenAI SDK with `OpenAIInstrumentor()`

2. **Agent Tracing**:
   - Instrument Azure AI Agents SDK with `AIAgentsInstrumentor()`
   - Wrap operations in custom spans for better visibility
   - Trace agent creation, execution, and file search operations

3. **View Traces**:
   - Access traces in Azure AI Foundry portal (Tracing tab)
   - Monitor LLM calls, agent operations, and tool usage
   - Debug issues with detailed span attributes

## Key Takeaways

✅ **Always configure Azure Monitor first** before making API calls
✅ **Use AIAgentsInstrumentor()** for agent tracing
✅ **Wrap operations in spans** for custom tracing
✅ **Set content recording** to capture prompts/responses (be mindful of PII)
✅ **Traces appear within 2-5 minutes** in the Azure AI Foundry portal

## Next Steps

- Explore the **Tracing tab** in Azure AI Foundry to see your traces
- Add custom spans to trace specific business logic
- Use traces to debug and optimize your AI applications
- Integrate evaluation loops with your traces
- Set up continuous monitoring for production workloads

## Official Documentation

- [Trace Applications](https://learn.microsoft.com/en-us/azure/ai-foundry/how-to/develop/trace-application)
- [Trace Agents SDK](https://learn.microsoft.com/en-us/azure/ai-foundry/how-to/develop/trace-agents-sdk)
- [Azure AI Foundry Observability](https://learn.microsoft.com/en-us/azure/ai-foundry/concepts/observability)

> 🏋️ **Health Reminder**: The LLM's suggestions are for demonstration only. For real health decisions, consult a professional.

Happy Observing & Tracing! ?